# **Outil de création d'une carte avec plusieurs parcours**

Les outils de recherches d'informations permettent de récupérer les emplacements de chaque lieu (coordonnées GPS), le type de lieu dans lequel il est catégorisé ainsi qu'une brève description du lieu. La récupération des coordonnées GPS permet de retracer les parcours. Il est possible à la fin du notebook, d'ajouter son propre fond de carte.

Exemple d'utilisation : deux chemin cités dans deux chapitres différentes d'un ouvrage. 

*Document d'entrée* : deux fichiers en format .txt contenant la liste des noms de lieux à recenser sur la carte et à comparer.

*Documents de sortie* : pour chaque liste de lieux (.txt) : un fichier avec les identifiants Wikidata des lieux + un fichier sous la forme d'un tableau lisible avec Excel est créé avec les informations récoltées + le même fichier mais en format JSON pour une utilisation machine. Pour la comparaison des deux parcours : une carte téléchargée en format HTML où les parcours sont tracés de différentes couleurs.


In [ ]:
#@markdown # Connection du notebook à son compte Google Drive et signalement du dossier de travail :

#@markdown - Lancer la cellule
#@markdown - Cliquer sur « Exécuter malgré tout » lors de l’apparition du message d’avertissement indiquant que le notebook n’a pas été créé par Google
#@markdown - Cliquer sur « Se connecter à Google Drive » lors de l’apparition du second message d’avertissement pour donner l’autorisation au notebook d’accéder à vos fichiers Google Drive
#@markdown - Choisir son compte Gmail puis cliquer sur « Autoriser »

!pip install geopy==1.17.0
!pip install wptools==0.4.17
from google.colab import drive
import folium
from geopy.geocoders import Nominatim
import pandas as pd
from folium import plugins
import json
import urllib.parse, urllib.request
import numpy as np
import csv
from collections import Counter
from branca.element import Figure
import os
import wptools
import re
import itertools

drive.mount('/content/drive/')
%cd /content/drive/My Drive/

#@markdown ###Indiquer le chemin vers le dossier de travail sur le Google Drive (si le dossier n'existe pas, il sera créé lors du lancement de la cellule) :
chemin_vers_le_dossier_de_travail = '/content/drive/My Drive/Visualisations lieux/'#@param {type:"string"}

if not os.path.exists(chemin_vers_le_dossier_de_travail):
      os.makedirs(chemin_vers_le_dossier_de_travail)
os.chdir(chemin_vers_le_dossier_de_travail)

In [ ]:
#@markdown # Récupération des identifiants Wikidata des lieux pour le premier fichier :
#@markdown ####Indiquer le chemin vers le fichier texte avec la liste des lieux à détecter :
chemin_liste_des_lieux = '/content/drive/My Drive/Visualisations lieux/lieux_1.txt'#@param {type:"string"}
#@markdown ####Changer la langue si nécessaire (par défaut : français) :
langue = "fr" #@param {type:"string"}
#@markdown ####Si la détection des lieux n'est pas assez précise, descendre le seuil à 0.8 (moins de noms vont être identifiés). Dans le cas contraire, monter le seuil à 1.
seuil = 0.8 #@param {type:"string"}

def CallWikifier(text, lang=langue, threshold=seuil): 
  # Thresold = taux de sureté. 0.8 = sûr que le résultat est exact; mais peu de résultat.
  # 1 = tous les résultats mais certains faux résultats
    # Url de demande:
    data = urllib.parse.urlencode([
        ("text", text), ("lang", lang),
        ("userKey", "nqvsutgqswvfmrcvyxjtopvpiukjtp"),
        ("pageRankSqThreshold", "%g" % threshold), ("applyPageRankSqThreshold", "true"),
        ("nTopDfValuesToIgnore", "200"), ("nWordsToIgnoreFromList", "200"),
        ("wikiDataClasses", "true"), ("wikiDataClassIds", "false"),
        ("support", "true"), ("ranges", "false"), ("minLinkFrequency", "4"),
        ("includeCosines", "false"), ("maxMentionEntropy", "3")
        ])
    url = "http://www.wikifier.org/annotate-article"
    # Appel de Wikifier 
    req = urllib.request.Request(url, data=data.encode("utf8"), method="POST")
    with urllib.request.urlopen(req, timeout = 100) as f:
        response = f.read()
        response = json.loads(response.decode("utf8"))
        print(response)
    # Sortie des annotations:
    #@markdown ####Indiquer le nom du fichier créé avec la liste des identifiants Wikidata :
    nom_fichier_liste_identifiants = 'id_wikidata_lieux_1.txt' #@param {type:"string"}
    with open(nom_fichier_liste_identifiants, 'w') as outfile:
        for annotation in response["annotations"]:
          ## Récupération (uniquement de l'ID wikidata):
          itemid = annotation["wikiDataItemId"] 
          itemid2 = "".join([str(itemid)," "])
          outfile.write(itemid2)

with open (chemin_liste_des_lieux, "r") as myfile:
    liste=myfile.read()

CallWikifier(text=liste)

In [ ]:
#@markdown # Récupération des identifiants Wikidata des lieux pour le second fichier :
#@markdown ####Indiquer le chemin vers le fichier texte avec la liste des lieux à détecter :
chemin_liste_des_lieux = '/content/drive/My Drive/Visualisations lieux/lieux_2.txt'#@param {type:"string"}
#@markdown ####Changer la langue si nécessaire (par défaut : français) :
langue = "fr" #@param {type:"string"}
#@markdown ####Si la détection des lieux n'est pas assez précise, descendre le seuil à 0.8 (moins de noms vont être identifiés). Dans le cas contraire, monter le seuil à 1.
seuil = 0.8 #@param {type:"string"}

def CallWikifier(text, lang=langue, threshold=seuil): 
  # Thresold = taux de sureté. 0.8 = sûr que le résultat est exact; mais peu de résultat.
  # 1 = tous les résultats mais certains faux résultats
    # Url de demande:
    data = urllib.parse.urlencode([
        ("text", text), ("lang", lang),
        ("userKey", "nqvsutgqswvfmrcvyxjtopvpiukjtp"),
        ("pageRankSqThreshold", "%g" % threshold), ("applyPageRankSqThreshold", "true"),
        ("nTopDfValuesToIgnore", "200"), ("nWordsToIgnoreFromList", "200"),
        ("wikiDataClasses", "true"), ("wikiDataClassIds", "false"),
        ("support", "true"), ("ranges", "false"), ("minLinkFrequency", "4"),
        ("includeCosines", "false"), ("maxMentionEntropy", "3")
        ])
    url = "http://www.wikifier.org/annotate-article"
    # Appel de Wikifier 
    req = urllib.request.Request(url, data=data.encode("utf8"), method="POST")
    with urllib.request.urlopen(req, timeout = 100) as f:
        response = f.read()
        response = json.loads(response.decode("utf8"))
        print(response)
    # Sortie des annotations:
        #@markdown ####Indiquer le nom du fichier créé avec la liste des identifiants Wikidata :
    nom_fichier_liste_identifiants = 'id_wikidata_lieux_2.txt' #@param {type:"string"}
    with open(nom_fichier_liste_identifiants, 'w') as outfile:
        for annotation in response["annotations"]:
          ## Récupération (uniquement de l'ID wikidata):
          itemid = annotation["wikiDataItemId"] 
          itemid2 = "".join([str(itemid)," "])
          outfile.write(itemid2)

with open (chemin_liste_des_lieux, "r") as myfile2:
    liste=myfile2.read()

CallWikifier(text=liste)

In [ ]:
#@markdown # Récupération des informations de chaque lieu reconnu et lié à Wikidata dans le premier fichier :
#@markdown Toutes les informations (le type de lieu, les coordonnées GPS (latitude et longitude) et une brève description du lieu), sont enregistrées et exportées sous la forme d'un fichier de données JSON et sous la forme d'un tableau dans un fichier Excel dans le Google Drive.

donnees1 = {}
nom1 = []
type_lieu = []
latitude = []
longitude = []
longitude = []
description = []

#@markdown ####Indiquer le nom du fichier avec les identifiants Wikidata créé dans la cellule précédente :
nom_fichier_liste_identifiants = "id_wikidata_lieux_1.txt" #@param {type:"string"}
#@markdown ####Indiquer le nom du fichier en format JSON créé avec les informations sur les lieux :
fichier_json_infos_lieux = "donnees_lieux_1.json" #@param {type:"string"}

# Utilisation du document créé précédement avec les identifiants Wikidata :
with open(nom_fichier_liste_identifiants, "r") as a_file:
      with open(fichier_json_infos_lieux, "w") as fichier: 
        for line in a_file:
            for word in line.split():
              page = wptools.page(wikibase=word, lang="fr")
              page.get_wikidata()
              page.get_parse()
## Pour avoir toutes les informations :
              try:
                infobox=page.data['infobox']
                nom1 = infobox.get("nom")
                type_lieu=infobox.get("type")
                if type_lieu is None: 
                  try:
                    type_lieu=page.data['what']
                  except KeyError as ke:
                    pass
                latitude = infobox.get("latitude")
                longitude = infobox.get("longitude")
                description = page.data["description"]
              except AttributeError as at:
                print("Ce n'est pas un lieu!")
# Création d'un dictionnaire + fichier json avec les informations récupérées sur les lieux :
              try:
                donnees1[nom1] = {"Type de lieu" : type_lieu,
                              "Latitude" : latitude, "Longitude" : longitude, "Description": description}
              except TypeError as tpy:
                pass
      
        json.dump(donnees1, fichier, ensure_ascii=False)

#@markdown ####Indiquer le nom du fichier en format XLSX créé avec les informations sur les lieux :
fichier_xlsx_infos_lieux = "informations_lieux_1.xlsx" #@param {type:"string"}

# Enregistrement de toutes les informations récupérées sous la forme d'un tableau Excel :
df = pd.DataFrame(donnees1)
df2 = df.transpose()
df2.to_excel(fichier_xlsx_infos_lieux)

In [ ]:
#@markdown # Récupération des informations de chaque lieu reconnu et lié à Wikidata dans le second fichier :
#@markdown Toutes les informations (le type de lieu, les coordonnées GPS (latitude et longitude) et une brève description du lieu), sont enregistrées et exportées sous la forme d'un fichier de données JSON et sous la forme d'un tableau dans un fichier Excel dans le Google Drive.

donnees2 = {}
nom2 = []
type_lieu = []
latitude = []
longitude = []
longitude = []
description = []

#@markdown ####Indiquer le nom du fichier avec les identifiants Wikidata créé dans la cellule précédente :
nom_fichier_liste_identifiants = "id_wikidata_lieux_2.txt" #@param {type:"string"}
#@markdown ####Indiquer le nom du fichier en format JSON créé avec les informations sur les lieux :
fichier_json_infos_lieux = "donnees_lieux_2.json" #@param {type:"string"}

# Utilisation du document créé précédement avec les identifiants Wikidata :
with open(nom_fichier_liste_identifiants, "r") as a_file:
      with open(fichier_json_infos_lieux, "w") as fichier: 
        for line in a_file:
            for word in line.split():
              page = wptools.page(wikibase=word, lang="fr")
              page.get_wikidata()
              page.get_parse()
## Pour avoir toutes les informations :
              try:
                infobox=page.data['infobox']
                nom2 = infobox.get("nom")
                type_lieu=infobox.get("type")
                if type_lieu is None: 
                  try:
                    type_lieu=page.data['what']
                  except KeyError as ke:
                    pass
                latitude = infobox.get("latitude")
                longitude = infobox.get("longitude")
                description = page.data["description"]
              except AttributeError as at:
                print("Ce n'est pas un lieu!")
# Création d'un dictionnaire + fichier json avec les informations récupérées sur les lieux :
              try:
                donnees2[nom2] = {"Type de lieu" : type_lieu,
                              "Latitude" : latitude, "Longitude" : longitude, "Description": description}
              except TypeError as tpy:
                pass
      
        json.dump(donnees2, fichier, ensure_ascii=False)

#@markdown ####Indiquer le nom du fichier en format XLSX créé avec les informations sur les lieux :
fichier_xlsx_infos_lieux = "informations_lieux_2.xlsx" #@param {type:"string"}

# Enregistrement de toutes les informations récupérées sous la forme d'un tableau Excel :
df = pd.DataFrame(donnees2)
df2 = df.transpose()
df2.to_excel(fichier_xlsx_infos_lieux)

In [ ]:
#@markdown # Création d'une carte pour comparer les deux parcours :

keys_lat = donnees1[nom1]["Latitude"]
keys_long = donnees1[nom1]["Longitude"]
liste_des_lat = []
liste_des_long = []

#@markdown ####Indiquer le nom du fichier en format JSON créé pour le premier fichier avec les informations sur les lieux :
fichier_json_infos_lieux_1 = "donnees_lieux_1.json" #@param {type:"string"}

fichier_json = fichier_json_infos_lieux_1
with open(fichier_json, "r") as f :
  contenu_json = json.load(f)
  pairs = contenu_json.items()
  for key, value in pairs:
      test_lat=value["Latitude"]
      liste_des_lat.append(test_lat)
      test_long=value["Longitude"]
      liste_des_long.append(test_long)

coords = []
for ju, ij in zip(liste_des_lat, liste_des_long):
  if ju is None:
    pass  
  elif ij is None:
    pass
  else:
    coords.append([float(ju), float(ij)])
print(coords) ### Crée un array de coordonnées en suppirimant les None

#@markdown ####Indiquer le nom du fichier en format JSON créé pour le second fichier avec les informations sur les lieux :
fichier_json_infos_lieux_2 = "donnees_lieux_2.json" #@param {type:"string"}

keys_lat2 = donnees2[nom2]["Latitude"]
keys_long2 = donnees2[nom2]["Longitude"]
liste_des_lat2 = []
liste_des_long2 = []
fichier_json2 = fichier_json_infos_lieux_2
with open(fichier_json2, "r") as f2 :
  contenu_json2 = json.load(f2)
  pairs2 = contenu_json2.items()
  for key2, value2 in pairs2:
      test_lat2=value2["Latitude"]
      liste_des_lat2.append(test_lat2)
      test_long2=value2["Longitude"]
      liste_des_long2.append(test_long2)
coords2 = []
for ju2, ij2 in zip(liste_des_lat2, liste_des_long2):
  if ju2 is None:
    pass  
  elif ij2 is None:
    pass
  else:
    coords2.append([float(ju2), float(ij2)])
print(coords2) ### Crée un array de coordonnées en suppirimant les None

fig5=Figure(height=550,width=750)
m5=folium.Map(location=[48.856614, 2.3522219], zoom_start=12)
fig5.add_child(m5)

f1=folium.FeatureGroup("Parcours1")
f2=folium.FeatureGroup("Parcours2")
line_1=folium.vector_layers.PolyLine(coords,popup='<b>Parcours_test</b>',tooltip='Tour de France',color='blue',weight=4).add_to(f1)
line_2=folium.vector_layers.PolyLine(coords2,popup='<b>Parcours_test2</b>',tooltip='Tour de France2',color='red',weight=4).add_to(f2)
f1.add_to(m5)
f2.add_to(m5)
folium.LayerControl().add_to(m5)

#@markdown ####Indiquer le nom de la carte créée :
nom_carte = 'carte_comparaison_parcours.html'#@param {type:"string"}

# Téléchargement de la carte
m5.save(nom_carte)
m5

In [ ]:
#@markdown # Créer et ajouter son fond de carte :
#@markdown ####Si l'on veut ajouter son propre fond de carte personnalisé :
#@markdown - Création d’un compte sur : https://mapwarper.net/.
#@markdown - Téléchargement du fond de carte (IIIF recommandé, avec la meilleure résolution possible).
#@markdown - Sur Map Warper, dans l’onglet "Upload Map", il est possible de charger le fond de carte que l’on veut utiliser et d'indiquer toutes les métadonnées nécessaires.
#@markdown - Dans l’onglet "Rectify", il faut indiquer le plus de points identiques sur les deux cartes pour permettre la superposition du fond de carte chargé sur des données géolocalisées. Cliquer sur un point sur chaque carte, puis cliquer sur « add Control point » en dessous des cartes. Répéter l’opération le plus de fois possible pour obtenir un bon résultat. Dans l'encadré en dessous des deux fonds de carte, cliquer sur "Warp image" : le fond s'ajoute. Si le résultat n'est pas satisfaisant, il possible d'ajouter d'autres repères pour améliorer la qualité du nouveau fond de carte créé.
#@markdown - Se rendre dans l'onglet "Export". Dans "Map services", copier le lien entier se situant après "Tiles" (Google/OSM scheme):", débutant par http et se terminant par .png
#@markdown - Coller ensuite le lien ci-dessous :
fond_de_carte_cree = "https://mapwarper.net/maps/tile/54525/{z}/{x}/{y}.png" #@param {type:"string"}

#@markdown ####Indiquer le nom de la carte créée :
nom_carte = 'carte_comparaison_parcours_perso.html'#@param {type:"string"}

fig5=Figure(height=550,width=750)
m5=folium.Map(location=[48.856614, 2.3522219], zoom_start=12,
                 tiles=fond_de_carte_cree, attr="None")

keys_lat = donnees1[nom1]["Latitude"]
keys_long = donnees1[nom1]["Longitude"]
liste_des_lat = []
liste_des_long = []
fichier_json = fichier_json_infos_lieux_1
with open(fichier_json, "r") as f :
  contenu_json = json.load(f)
  pairs = contenu_json.items()
  print(pairs)
  for key, value in pairs:
      test_lat=value["Latitude"]
      liste_des_lat.append(test_lat)
      test_long=value["Longitude"]
      liste_des_long.append(test_long)

coords = []
for ju, ij in zip(liste_des_lat, liste_des_long):
  if ju is None:
    pass  
  elif ij is None:
    pass
  else:
    coords.append([float(ju), float(ij)])
#print(coords) ### Crée un array de coordonnées en suppirimant les None

keys_lat2 = donnees2[nom2]["Latitude"]
keys_long2 = donnees2[nom2]["Longitude"]
liste_des_lat2 = []
liste_des_long2 = []
fichier_json2 = fichier_json_infos_lieux_2
with open(fichier_json2, "r") as f2 :
  contenu_json2 = json.load(f2)
  pairs2 = contenu_json2.items()
  for key2, value2 in pairs2:
      test_lat2=value2["Latitude"]
      liste_des_lat2.append(test_lat2)
      test_long2=value2["Longitude"]
      liste_des_long2.append(test_long2)
coords2 = []
for ju2, ij2 in zip(liste_des_lat2, liste_des_long2):
  if ju2 is None:
    pass  
  elif ij2 is None:
    pass
  else:
    coords2.append([float(ju2), float(ij2)])
#print(coords2) ### Crée un array de coordonnées en suppirimant les None

f1=folium.FeatureGroup("Parcours1")
f2=folium.FeatureGroup("Parcours2")
line_1=folium.vector_layers.PolyLine(coords,popup='<b>Parcours_test</b>',tooltip='Tour de France',color='blue',weight=4).add_to(f1)
line_2=folium.vector_layers.PolyLine(coords2,popup='<b>Parcours_test2</b>',tooltip='Tour de France2',color='red',weight=4).add_to(f2)
f1.add_to(m5)
f2.add_to(m5)
folium.LayerControl().add_to(m5)

# Téléchargement de la carte
m5.save(nom_carte)
m5